In [1]:
import os, sys
sys.path.append("motion_generation")
sys.path.append("rig_agnostic_encoding/functions")
sys.path.append("rig_agnostic_encoding/models")

from motion_generation.MoE import MoE
import motion_generation
from motion_generation.GRU import GRU
from motion_generation.LSTM import LSTM
from motion_generation.MotionGeneration import MotionGenerationModel
from motion_generation.MotionGeneration_v2 import MotionGenerationModel as MotionGenerationModel_v2
from motion_generation.MotionGeneration_v3 import MotionGenerationModel as MotionGenerationModel_v3
from motion_generation.MotionGeneration_v4 import MotionGenerationModel as MotionGenerationModel_v4
from motion_generation.MotionGeneration_v4_Concat import MotionGenerationModel as MotionGenerationModel_Concat
from motion_generation.MotionGenerationRNN import MotionGenerationModelRNN
from motion_generation.MotionGenerationBatch import MotionGenerationModelBatch
from rig_agnostic_encoding.models.MLP import MLP
from rig_agnostic_encoding.models.MLP_v2 import MLP as MLP_v2

from rig_agnostic_encoding.models.DEC import DEC
from rig_agnostic_encoding.models.MLP_MIX import MLP_MIX
from rig_agnostic_encoding.models.MLP_MIX_cosine import MLP_MIX as MLP_MIX_COSINE

from rig_agnostic_encoding.models.RBF import RBF
import rig_agnostic_encoding.models.RBF as RB
from rig_agnostic_encoding.models.RBF_Ortho import RBF as RBF_ORTHO
from rig_agnostic_encoding.models.RBF_Ortho_v2 import RBF as RBF_ORTHO_v2


from rig_agnostic_encoding.models.VAE import VAE
from rig_agnostic_encoding.functions.DataProcessingFunctions import clean_checkpoints
from GlobalSettings import MODEL_PATH
import bz2
from cytoolz import concat, sliding_window, accumulate
from operator import add
from collections import OrderedDict
import torch
import torch.nn as nn
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
import func as F
import _pickle as pickle
import json as js
import importlib
import random

In [2]:
config = {
    "hidden_dim": 256,
    "k": 256,
    "z_dim": 256,
    "lr": 1e-4,
    "batch_size": 16,
    "keep_prob": .2,
    "loss_fn":torch.nn.functional.mse_loss,
    "optimizer":torch.optim.AdamW,
    "scheduler":torch.optim.lr_scheduler.StepLR,
    "scheduler_param": {"step_size":80, "gamma":.9},
    "basis_func":"gaussian",
    "n_centroid":64,
    "k_experts": 4,
    "gate_size": 128,
    "g_hidden_dim": 512,
    "num_layers": 4,
    "autoregress_prob":0,
    "autoregress_inc":0.3,
    "autoregress_ep":20,
    "autoregress_max_prob":1,
    "cost_hidden_dim":128,
    "seq_len":13,
    "device":"cuda"
    }

In [3]:
def getFilesNames(file_paths, data_path, MAX_FILES=-1):
    for dname, dirs, files in os.walk(data_path):
        for i, file in enumerate(files):
            file_paths.append(os.path.join(dname, file))
            if MAX_FILES > 0 and i >= MAX_FILES:
                break
    return file_paths

data_path = "/home/nuoc/Documents/MEX/data/data/Dataset_R1_Two_1"
data_path2 = "/home/nuoc/Documents/MEX/data/data/Dataset_R2_Two_1"
# data_path3 = "/home/nuoc/Documents/MEX/data/data/Dataset_R3_Two_1"
# data_path4 = "/home/nuoc/Documents/MEX/data/data/Dataset_R4_Two_1"
file_paths = getFilesNames([],data_path)
file_paths2 = getFilesNames([],data_path2)
# file_paths3 = getFilesNames([],data_path3)
# file_paths4 = getFilesNames([],data_path4)

print(len(file_paths))
print(len(file_paths2))





240
240


In [12]:
phase_features = ["phase_vec_l2"]
pose_features = ["pos", "rotMat2", "velocity"]
cost_features = ["posCost", "rotCost"]
pose_label_feature = ["chainPos", "isLeft", "geoDistanceNormalised"]
target_features = ["targetPosition", "targetRotation"]
features = phase_features + pose_features + cost_features + target_features
clips = []
feature_dims = {}


In [5]:
data = F.process_data_multithread(file_paths, features)
data2 = F.process_data_multithread(file_paths2, features)

2021-05-09 16:56:53,342	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265
2021-05-09 16:57:12,147	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265


In [6]:
pose_labels1 = F.process_data_multithread([file_paths[0]], pose_label_feature)
pose_labels2 = F.process_data_multithread([file_paths2[0]], pose_label_feature)
# data3 = F.process_data_multithread(file_paths3, features)
# data4 = F.process_data_multithread(file_paths4, features)

2021-05-09 16:57:32,276	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265
2021-05-09 16:57:37,323	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265


In [9]:
obj = {"data":data, "data2":data2, "pose_label1":pose_labels1, "pose_label2":pose_labels2}
F.save(obj, filename="transfer_learning_set_R1-R2_Two", path="/home/nuoc/Documents/MEX/data/")

In [3]:
obj = F.load("/home/nuoc/Documents/MEX/data/transfer_learning_set_R1-R2_Two.pbz2")
data = obj["data"]
data2 = obj["data2"]

In [4]:
pose_labels1 = obj["pose_label1"]
pose_labels2 = obj["pose_label2"]

In [13]:
feature_dims = data[0][1]
feature_dims2 = data2[0][1]
# feature_dims3 = data3[0][1]
# feature_dims4 = data4[0][1]

clips = [np.copy(d[0]) for d in data]
clips2 = [np.copy(d[0]) for d in data2]

pose_labels1 = pose_labels1[0]
pose_labels2 = pose_labels2[0]
# clips3 = [np.copy(d[0]) for d in data3]
# clips4 = [np.copy(d[0]) for d in data4]





In [14]:
phase_dim = sum([feature_dims[feature] for feature in phase_features])
pose_dim = sum([feature_dims[feature] for feature in pose_features])
pose_dim2 = sum([feature_dims2[feature] for feature in pose_features])
# pose_dim3 = sum([feature_dims3[feature] for feature in pose_features])
# pose_dim4 = sum([feature_dims4[feature] for feature in pose_features])
cost_dim = sum([feature_dims[feature] for feature in cost_features])
target_dim = sum([feature_dims[feature] for feature in target_features])
print(phase_dim, " ", cost_dim, " ", target_dim)
# print(pose_dim, pose_dim2, pose_dim3, pose_dim4)

8   24   48


In [15]:
x_tensors = torch.stack([F.normaliseT(torch.from_numpy(clip[:-1])).float() for clip in clips])
y_tensors = torch.stack([torch.from_numpy(clip[1:]).float() for clip in clips])

x_tensors2 = torch.stack([F.normaliseT(torch.from_numpy(clip[:-1])).float() for clip in clips2])
y_tensors2 = torch.stack([torch.from_numpy(clip[1:]).float() for clip in clips2])

# x_tensors3 = torch.stack([F.normaliseT(torch.from_numpy(clip[:-1])).float() for clip in clips3])
# y_tensors3 = torch.stack([torch.from_numpy(clip[1:]).float() for clip in clips3])
#
# x_tensors4 = torch.stack([F.normaliseT(torch.from_numpy(clip[:-1])).float() for clip in clips4])
# y_tensors4 = torch.stack([torch.from_numpy(clip[1:]).float() for clip in clips4])

pose_data1 = x_tensors[:,  :,  phase_dim:phase_dim+pose_dim]
pose_data2 = x_tensors2[:, :, phase_dim:phase_dim+pose_dim2]
# pose_data3 = x_tensors3[:, :, phase_dim:phase_dim+pose_dim3]
# pose_data4 = x_tensors4[:, :, phase_dim:phase_dim+pose_dim4]
# pose_data = torch.cat((pose_data1, pose_data2, pose_data3, pose_data4), dim=2)
pose_data = torch.cat((pose_data1, pose_data2), dim=2)

In [16]:
dataset_p = TensorDataset(pose_data, pose_data)
dataset_p1 = TensorDataset(pose_data1, pose_data1)
dataset_p2 = TensorDataset(pose_data2, pose_data2)
datasetR1 = TensorDataset(x_tensors, y_tensors)
datasetR2 = TensorDataset(x_tensors2, y_tensors2)
# datasetR3 = TensorDataset(x_tensors3, y_tensors3)
# datasetR4 = TensorDataset(x_tensors4, y_tensors4)
# datasetR1_R3 = TensorDataset(x_tensors, y_tensors3)

N = len(x_tensors)

train_ratio = int(.7*N)
val_ratio = int((N-train_ratio) / 2.0)
test_ratio = N - train_ratio - val_ratio

train_set_p, val_set_p, test_set_p = random_split(dataset_p, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(2021))
train_set_p1, val_set_p1, test_set_p1 = random_split(dataset_p1, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(2021))
train_set_p2, val_set_p2, test_set_p2 = random_split(dataset_p2, [val_ratio*2, val_ratio, train_ratio-test_ratio], generator=torch.Generator().manual_seed(2021))
train_setR1, val_setR1, test_setR1 = random_split(datasetR1, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(2021))
train_setR2, val_setR2, test_setR2 = random_split(datasetR2, [val_ratio*2, val_ratio, train_ratio-val_ratio], generator=torch.Generator().manual_seed(2021))
# train_setR3, val_setR3, test_setR3 = random_split(datasetR3, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(2021))
# train_setR4, val_setR4, test_setR4 = random_split(datasetR4, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(2021))
# train_setR1_3, val_setR1_3, test_setR1_3 = random_split(datasetR1_R3, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(2021))
# print(len(train_set), len(val_set), len(test_set))
# print(len(train_set2), len(val_set2), len(test_set2))

In [17]:
def extract_targets(train_set, val_set, test_set, target_dim):
    t1, t2, t3, t33 = [], [], [], []
    for i in range(len(train_set)):
        x = train_set[i][0]
        y = train_set[i][1]
        t1.append((x[:, :-target_dim], y[:, :-target_dim]))
    for i in range(len(val_set)):
        x = val_set[i][0]
        y = val_set[i][1]
        t2.append((x[:, :-target_dim:], y[:, :-target_dim:]))
    for i in range(len(test_set)):
        x = test_set[i][0]
        y = test_set[i][1]
        t3.append((x[:, :-target_dim:], y[:, :-target_dim:]))
        t33.append((x[:, :-target_dim:], y))
    return t1, t2, t3, t33



In [18]:
t1, v1, te1, te11 = extract_targets(train_set=train_setR1, val_set=val_setR1, test_set=test_setR1, target_dim=target_dim)
t2, v2, te2, te22 = extract_targets(train_set=train_setR2, val_set=val_setR2, test_set=test_setR2, target_dim=target_dim)
# t3, v3, te3, te33= extract_targets(train_set=train_setR3, val_set=val_setR3, test_set=test_setR3, target_dim=target_dim)
# t4, v4, te4, te44 = extract_targets(train_set=train_setR4, val_set=val_setR4, test_set=test_setR4, target_dim=target_dim)
# t1_3, v1_3, te1_3, te1_33 = extract_targets(train_set=train_setR1_3, val_set=val_setR1_3, test_set=test_setR1_3, target_dim=target_dim)



In [8]:

class MLP(pl.LightningModule):
    def __init__(self, config:dict=None, dimensions:list=None, pose_labels=None,
                 train_set=None, val_set=None, test_set=None, pos_dim=0, rot_dim=0, vel_dim=0,
                 name:str="model", single_module:int=0, save_period=5,
                 workers=6):

        super(MLP, self).__init__()
        self.name = name
        self.dimensions = dimensions
        self.single_module = single_module
        self.act = nn.ELU
        self.save_period = save_period
        self.workers = workers
        self.config=config

        self.pos_dim = pos_dim
        self.rot_dim = pos_dim + rot_dim
        self.vel_dim = self.rot_dim + vel_dim

        self.hidden_dim = config["hidden_dim"]
        self.keep_prob = config["keep_prob"]
        self.k = config["k"]
        self.learning_rate = config["lr"]
        self.batch_size = config["batch_size"]

        self.dimensions = dimensions if len(dimensions) > 1 else \
            [dimensions[0], self.hidden_dim, self.hidden_dim, self.k]

        self.loss_fn = config["loss_fn"] if "loss_fn" in config else nn.functional.mse_loss
        self.opt = config["optimizer"] if "optimizer" in config else torch.optim.Adam
        self.scheduler = config["scheduler"] if "scheduler" in config else None
        self.scheduler_param = config["scheduler_param"] if "scheduler_param" in config else None

        self.pose_labels = pose_labels  # should be Tensor(1,63) for example
        self.use_label = pose_labels is not None

        self.train_set, self.val_set, self.test_set = train_set, val_set, test_set
        self.best_val_loss = np.inf

        self.encoder, self.decoder = nn.Module(), nn.Module()
        self.build()
        if "device" not in config:
            config["device"] = "cuda"

        self.encoder.to(config["device"])
        self.decoder.to(config["device"])

        self.encoder.apply(self.init_params)
        self.decoder.apply(self.init_params)

    def build(self):
        layer_sizes = list(sliding_window(2, self.dimensions))
        if self.single_module == -1 or self.single_module == 0:
            layers = []
            for i, size in enumerate(layer_sizes):
                layers.append(("fc"+str(i), nn.Linear(size[0], size[1])))
                if i < len(self.dimensions)-2:
                    layers.append(("act"+str(i), self.act()))
                if i == 0:
                    layers.append(("drop", nn.Dropout(self.keep_prob)))
            self.encoder = nn.Sequential(OrderedDict(layers))
        else:
            self.encoder = nn.Sequential()

        if self.single_module == 0 or self.single_module == 1:
            layers = []
            if self.pose_labels is not None:
                layer_sizes[-1] = (layer_sizes[-1][0], layer_sizes[-1][1] + self.pose_labels.numel())
            for i, size in enumerate(layer_sizes[-1::-1]):
                layers.append(("fc"+str(i), nn.Linear(size[1], size[0])))
                if i < len(self.dimensions)-2:
                    layers.append(("act"+str(i), self.act()))
                if i == 0:
                    layers.append(("drop", nn.Dropout(self.keep_prob)))
            self.decoder = nn.Sequential(OrderedDict(layers))
        else:
            self.decoder = nn.Sequential()


    def forward(self, x:torch.Tensor) -> torch.Tensor:
        return self.decode(self.encode(x))

    def encode(self, x):
        return self.encoder(x)

    def decode(self, h):
        if self.use_label:
            h = torch.cat((h, self.pose_labels.expand(h.shape[0],-1)), dim=1)
        return self.decoder(h)

    def decode_label(self, h):
        h = torch.cat((h, self.pose_labels.expand(h.shape[0],-1)), dim=1)
        return self.decoder(h)

    def loss(self, x, y):
        recon_loss = self.loss_fn(x, y)
        pos_loss = self.loss_fn(x[:, :self.pos_dim], y[:, :self.pos_dim])
        rot_loss = self.loss_fn(x[:, self.pos_dim:self.rot_dim], y[:, self.pos_dim:self.rot_dim])
        return recon_loss, pos_loss, rot_loss

    def training_step(self, batch, batch_idx):
        x, y = batch
        prediction = self(x)
        recon_loss, pos_loss, rot_loss = self.loss(prediction, y)

        self.log("ptl/train_loss", recon_loss)
        self.log("ptl/train_pos_loss", pos_loss)
        self.log("ptl/train_rot_loss", rot_loss)
        return recon_loss

    def validation_step(self, batch, batch_idx):
        x, y = batch

        prediction = self(x)
        recon_loss, pos_loss, rot_loss = self.loss(prediction, y)

        self.log("ptl/val_loss", recon_loss, prog_bar=True)
        self.log("ptl/val_pos_loss", pos_loss, prog_bar=True)
        self.log("ptl/val_rot_loss", rot_loss, prog_bar=True)
        return {"val_loss":recon_loss}

    def test_step(self, batch, batch_idx):
        x, y = batch

        prediction = self(x)
        recon_loss, pos_loss, rot_loss = self.loss(prediction, y)

        self.log("ptl/test_loss", recon_loss)
        self.log("ptl/test_pos_loss", pos_loss)
        self.log("ptl/test_rot_loss", rot_loss)
        return {"test_loss":recon_loss}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        self.log("avg_val_loss", avg_loss)
        if avg_loss < self.best_val_loss:
            self.best_val_loss = avg_loss
            if self.current_epoch % self.save_period == 0:
                self.save_checkpoint(best_val_loss=self.best_val_loss.item())

    def save_checkpoint(self, best_val_loss:float=np.inf, checkpoint_dir=MODEL_PATH):
        config = {
            "hidden_dim":self.hidden_dim,
            "k":self.k,
            "lr":self.learning_rate,
            "batch_size":self.batch_size,
            "keep_prob":self.keep_prob,
            "optimizer":self.opt,
            "scheduler":self.scheduler,
            "scheduler_param":self.scheduler_param,
            "device":self.config["device"],
            "dims": [self.pos_dim, self.rot_dim, self.vel_dim]
        }
        model = {"config":config, "name":self.name,"dimensions":self.dimensions,
                 "pose_labels": self.pose_labels,
                 "single_module":self.single_module,
                 "encoder":self.encoder.state_dict(),
                 "decoder":self.decoder.state_dict()}

        if not os.path.exists(checkpoint_dir):
            os.mkdir(checkpoint_dir)
        path = os.path.join(checkpoint_dir, self.name)
        if not os.path.exists(path):
            os.mkdir(path)

        filePath = os.path.join(path, str(best_val_loss)+"."+str(self.k)+".pbz2")
        with bz2.BZ2File(filePath, "w") as f:
            pickle.dump(model, f)
        return filePath

    @staticmethod
    def load_checkpoint(filePath):
        with bz2.BZ2File(filePath, "rb") as f:
            obj = pickle.load(f)
        model = MLP(config=obj["config"], single_module=obj["single_module"], pose_labels=obj["pose_labels"],
                    name=obj["name"], dimensions=obj["dimensions"])

        model.encoder.load_state_dict(obj["encoder"])
        model.decoder.load_state_dict(obj["decoder"])
        model.pos_dim = obj["config"]["dims"][0]
        model.rot_dim = obj["config"]["dims"][1]
        model.vel_dim = obj["config"]["dims"][2]
        return model

    def freeze(self, flag=False):
        self.encoder.requires_grad_(flag)
        self.decoder.requires_grad_(flag)

    def configure_optimizers(self):
        optimizer = self.opt(self.parameters(), lr=self.learning_rate)
        if self.scheduler is not None:
            scheduler = self.scheduler(optimizer, **self.scheduler_param)
            return [optimizer], [scheduler]
        return optimizer

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size, pin_memory=True, num_workers=self.workers)

    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.batch_size, pin_memory=True, num_workers=self.workers)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size, pin_memory=True, num_workers=self.workers)

    @staticmethod
    def init_params(m):
        if type(m) == nn.Linear:
            nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(.01)


In [9]:
ae_name1 = "AE_R1"
# ae1 = MLP_v2(config=config, dimensions=[pose_dim],
#                  pos_dim=feature_dims["pos"], rot_dim=feature_dims["rotMat2"], vel_dim=feature_dims["velocity"],
#                  train_set=train_set_p1, val_set=val_set_p1, test_set=test_set_p1, name=ae_name1)

ae1 = MLP.load_checkpoint("/home/nuoc/Documents/MEX/models/version_0.2/AE_R1/0.001.256.pbz2")

In [10]:
ae_name2 = "AE_R2"
# ae2 = MLP_v2(config=config, dimensions=[pose_dim2],
#                  pos_dim=feature_dims2["pos"], rot_dim=feature_dims2["rotMat2"], vel_dim=feature_dims2["velocity"],
#                  train_set=train_set_p2, val_set=val_set_p2, test_set=test_set_p2, name=ae_name2)

ae2 = MLP.load_checkpoint("/home/nuoc/Documents/MEX/models/version_0.2/AE_R2/0.001.256.pbz2")

In [35]:
MAX_EPOCHS = 200

checkpoint_callback = ModelCheckpoint(monitor="avg_val_loss", save_top_k=3)
earlystopping = EarlyStopping(monitor="avg_val_loss", patience=10)
logger=TensorBoardLogger(save_dir="logs/", name=ae_name1, version="0.1")

trainer = pl.Trainer(
    default_root_dir="/home/nuoc/Documents/MEX/src/motion_generation/checkpoints",
    gpus=1, precision=16,
    callbacks=[earlystopping],
    min_epochs=20,
    logger=logger,
    max_epochs=MAX_EPOCHS,
    stochastic_weight_avg=True
)


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.


In [36]:
# trainer.fit(ae1)
trainer.fit(ae2)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 227 K 
1 | decoder | Sequential | 227 K 
---------------------------------------
454 K     Trainable params
0         Non-trainable params
454 K     Total params
1.817     Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7fab19d6f100> for <torch.optim.swa_utils.SWALR object at 0x7fab19a36e50>
  warnings.warn(*args, **kwargs)


1

In [37]:
# trainer.test(ae1)
trainer.test(ae2)



LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.0992501974105835,
 'ptl/test_pos_loss': 0.18037962913513184,
 'ptl/test_rot_loss': 0.06784066557884216,
 'test_loss': 0.12275812774896622}
--------------------------------------------------------------------------------


[{'test_loss': 0.12275812774896622,
  'ptl/test_loss': 0.0992501974105835,
  'ptl/test_pos_loss': 0.18037962913513184,
  'ptl/test_rot_loss': 0.06784066557884216}]

In [148]:
ae1.save_checkpoint(best_val_loss=0.001)
ae2.save_checkpoint(best_val_loss=0.001)

'/home/nuoc/Documents/MEX/models/version_0.2/AE_R2/0.001.256.pbz2'

In [24]:
class RBF(pl.LightningModule):
    def __init__(self, config:dict=None, input_dims:list=None, pose_labels=None,
                 train_set=None, val_set=None, test_set=None,
                 name:str="model", save_period=5, workers=6):

        super(RBF, self).__init__()

        M = len(input_dims)

        self.name = name
        self.input_dims = input_dims
        self.input_slice = [0] + list(accumulate(add, input_dims))

        self.act = nn.ELU
        self.save_period = save_period
        self.workers = workers
        self.pose_labels = pose_labels if pose_labels is not None else [None for _ in range(M)]

        self.config = config
        self.basis_func = RB.basis_func_dict()[config["basis_func"]]
        self.hidden_dim = config["hidden_dim"]
        self.keep_prob = config["keep_prob"]
        self.k = config["k"]
        self.z_dim = config["z_dim"]

        self.learning_rate = config["lr"]
        self.batch_size = config["batch_size"]

        self.loss_fn = config["loss_fn"] if "loss_fn" in config else nn.functional.mse_loss
        self.opt = config["optimizer"] if "optimizer" in config else torch.optim.Adam
        self.scheduler = config["scheduler"] if "scheduler" in config else None
        self.scheduler_param = config["scheduler_param"] if "scheduler_param" in config else None

        self.models = [MLP_v2(config=config, dimensions=[input_dims[i]], pose_labels=self.pose_labels[i],
                           name="M"+str(i), single_module=0) for i in range(M)]
        self.active_models = self.models

        self.cluster_model = RB.RBF_Layer(in_features=self.k, out_features=self.z_dim, basis_func=self.basis_func)

        self.train_set = train_set
        self.val_set = val_set
        self.test_set = test_set

        self.best_val_loss = np.inf

    def forward(self, x:torch.Tensor) -> torch.Tensor:
        x_tensors = [x[:, d0:d1] for d0, d1 in zip(self.input_slice[:-1], self.input_slice[1:])]

        encoded = [m.encode(x_tensors[i]) for i, m in enumerate(self.active_models)]
        embeddings = [self.cluster_model(vec) for vec in encoded]
        decoded = [m.decode(embeddings[i]) for i, m in enumerate(self.active_models)]

        return decoded

    def training_step(self, batch, batch_idx):
        x, y = batch

        x = x.view(-1, x.shape[-1])
        y = y.view(-1, y.shape[-1])
        prediction = self(x)
        y_tensors = [y[:, d0:d1] for d0, d1 in zip(self.input_slice[:-1], self.input_slice[1:])]
        losses = [self.active_models[i].loss(prediction[i], y_tensors[i])[0] for i in range(len(prediction))]

        loss = sum(losses) / float(len(losses))
        self.log("ptl/train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch

        x = x.view(-1, x.shape[-1])
        y = y.view(-1, y.shape[-1])
        prediction = self(x)
        y_tensors = [y[:, d0:d1] for d0, d1 in zip(self.input_slice[:-1], self.input_slice[1:])]
        losses = [self.active_models[i].loss(prediction[i], y_tensors[i])[0] for i in range(len(prediction))]

        loss = sum(losses) / float(len(losses))

        self.log('ptl/val_loss', loss, prog_bar=True)
        return {"val_loss":loss}

    def test_step(self, batch, batch_idx):
        x, y = batch

        x = x.view(-1, x.shape[-1])
        y = y.view(-1, y.shape[-1])
        prediction = self(x)
        y_tensors = [y[:, d0:d1] for d0, d1 in zip(self.input_slice[:-1], self.input_slice[1:])]
        losses = [self.active_models[i].loss(prediction[i], y_tensors[i])[0] for i in range(len(prediction))]

        loss = sum(losses) / float(len(losses))

        self.log('ptl/test_loss', loss, prog_bar=True)
        return {"test_loss":loss}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        self.log("avg_val_loss", avg_loss)
        if avg_loss < self.best_val_loss:
            self.best_val_loss = avg_loss
            if self.current_epoch % self.save_period == 0:
                self.save_checkpoint(best_val_loss=self.best_val_loss.item())

    def save_checkpoint(self, best_val_loss:float=np.inf, checkpoint_dir=MODEL_PATH):
        path = os.path.join(checkpoint_dir, self.name)

        config = {
            "hidden_dim": self.hidden_dim,
            "k": self.k,
            "lr": self.learning_rate,
            "batch_size": self.batch_size,
            "keep_prob":self.keep_prob,
            "optimizer": self.opt,
            "scheduler": self.scheduler,
            "scheduler_param": self.scheduler_param,
            "basis_func":self.config["basis_func"],
        }

        model_paths = [m.save_checkpoint(best_val_loss=best_val_loss, checkpoint_dir=path) for m in self.models]
        model = {"config":config, "name":self.name, "model_paths":model_paths,
                 "input_dims":self.input_dims, "pose_labels":self.pose_labels,
                 "cluster_model":self.cluster_model.state_dict()
                 }

        if not os.path.exists(checkpoint_dir):
            os.mkdir(checkpoint_dir)
        path = os.path.join(checkpoint_dir, self.name)
        if not os.path.exists(path):
            os.mkdir(path)

        filePath = os.path.join(path, str(best_val_loss)+"."+str(self.k)+".pbz2")
        with bz2.BZ2File(filePath, "w") as f:
            pickle.dump(model, f)
        return filePath

    @staticmethod
    def load_checkpoint(filePath):
        with bz2.BZ2File(filePath, "rb") as f:
            obj = pickle.load(f)
        model = RBF(config=obj["config"], name=obj["name"],
                    input_dims=obj["input_dims"], pose_labels=obj["pose_labels"])

        models = [MLP.load_checkpoint(path) for path in obj["model_paths"]]
        model.models = models
        model.cluster_model.load_state_dict(obj["cluster_model"])
        return model

    def configure_optimizers(self):
        optimizer = self.opt(self.parameters(), lr=self.learning_rate)
        if self.scheduler is not None:
            scheduler = self.scheduler(optimizer, **self.scheduler_param)
            return [optimizer], [scheduler]
        return optimizer

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size, pin_memory=True, num_workers=self.workers)

    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.batch_size, pin_memory=True, num_workers=self.workers)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size, pin_memory=True, num_workers=self.workers)

    @staticmethod
    def init_params(m):
        if type(m) == nn.Linear:
            nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(.01)

    def add_models(self, input_dims:list=None, pose_labels:list=None, freeze=False):
        n = len(self.models)+1
        if pose_labels is not None:
            self.models += [MLP(config=self.config, dimensions=[input_dims[i]], pose_labels=pose_labels[i],
                            name="M" + str(i+n), single_module=0) for i in range(len(input_dims))]
        else:
            self.models += [MLP(config=self.config, dimensions=[input_dims[i]],
                            name="M" + str(i+n), single_module=0) for i in range(len(input_dims))]
        if freeze:
            for model in self.active_models:
                model.freeze(True)
            self.active_models = self.models[n-1:]
            self.input_dims = input_dims
        else:
            self.active_models = self.models
            self.input_dims += input_dims

        self.input_slice = [0] + list(accumulate(add, self.input_dims))

class RBF_Layer(nn.Module):
    """
       from JeremyLinux on GitHub {https://github.com/JeremyLinux/PyTorch-Radial-Basis-Function-Layer/blob/master/Torch%20RBF/torch_rbf.py}

       Transforms incoming data using a given radial basis function:
       u_{i} = rbf(||x - c_{i}|| / s_{i})
       Arguments:
           in_features: size of each input sample
           out_features: size of each output sample
       Shape:
           - Input: (N, in_features) where N is an arbitrary batch size
           - Output: (N, out_features) where N is an arbitrary batch size
       Attributes:
           centres: the learnable centres of shape (out_features, in_features).
               The values are initialised from a standard normal distribution.
               Normalising inputs to have mean 0 and standard deviation 1 is
               recommended.

           sigmas: the learnable scaling factors of shape (out_features).
               The values are initialised as ones.

           basis_func: the radial basis function used to transform the scaled
               distances.
       """

    def __init__(self, in_features: int=0, out_features: int=0, basis_func=None, device="cuda"):
        super(RBF_Layer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.centres = nn.Parameter(torch.Tensor(out_features, in_features))
        self.sigmas = nn.Parameter(torch.Tensor(out_features))
        self.basis_func = basis_func
        self.device = device
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.normal_(self.centres, 0, 1)
        nn.init.constant_(self.sigmas, .01)

    def forward(self, x):
        size = (x.size(0), self.out_features, self.in_features)
        x = x.unsqueeze(1).expand(size)
        c = self.centres.unsqueeze(0).expand(size)
        distances = (x - c).pow(2).sum(-1).pow(0.5) * self.sigmas.unsqueeze(0)  # ALT. / (2*sigma**2)
        return self.basis_func(distances)

    def freeze(self, flag=False):
        self.centres.requires_grad = flag
        self.sigmas.requires_grad = flag

    def loss(self):
        return torch.nn.functional.mse_loss(self.centres @ self.centres.T, torch.diag(torch.ones(self.out_features, device=self.device)))
        # return torch.mean((1 - (self.centres @ self.centres.T).sum(dim=1))**2)



class RBF_Ortho(pl.LightningModule):
    def __init__(self, config:dict=None, input_dims:list=None, pose_labels=None,
                 train_set=None, val_set=None, test_set=None,
                 name:str="model", save_period=5, workers=6):

        super(RBF_Ortho, self).__init__()

        M = len(input_dims)

        self.name = name
        self.input_dims = input_dims
        self.input_slice = [0] + list(accumulate(add, input_dims))

        self.act = nn.ELU
        self.save_period = save_period
        self.workers = workers
        self.pose_labels = pose_labels if pose_labels is not None else [None for _ in range(M)]

        self.config = config
        self.basis_func = RB.basis_func_dict()[config["basis_func"]]
        self.hidden_dim = config["hidden_dim"]
        self.keep_prob = config["keep_prob"]
        self.k = config["k"]
        self.z_dim = config["z_dim"]

        self.learning_rate = config["lr"]
        self.batch_size = config["batch_size"]

        self.loss_fn = config["loss_fn"] if "loss_fn" in config else nn.functional.mse_loss
        self.opt = config["optimizer"] if "optimizer" in config else torch.optim.Adam
        self.scheduler = config["scheduler"] if "scheduler" in config else None
        self.scheduler_param = config["scheduler_param"] if "scheduler_param" in config else None

        self.models = [MLP_v2(config=config, dimensions=[input_dims[i]], pose_labels=self.pose_labels[i],
                           name="M"+str(i), single_module=0) for i in range(M)]
        self.active_models = self.models

        self.cluster_model = RBF_Layer(in_features=self.k, out_features=self.z_dim, basis_func=self.basis_func)

        self.train_set = train_set
        self.val_set = val_set
        self.test_set = test_set

        self.best_val_loss = np.inf

    def forward(self, x:torch.Tensor) -> torch.Tensor:
        x_tensors = [x[:, d0:d1] for d0, d1 in zip(self.input_slice[:-1], self.input_slice[1:])]

        encoded = [m.encode(x_tensors[i]) for i, m in enumerate(self.active_models)]
        embeddings = [self.cluster_model(vec) for vec in encoded]
        decoded = [m.decode(embeddings[i]) for i, m in enumerate(self.active_models)]

        return decoded

    def training_step(self, batch, batch_idx):
        x, y = batch

        x = x.view(-1, x.shape[-1])
        y = y.view(-1, y.shape[-1])
        prediction = self(x)
        y_tensors = [y[:, d0:d1] for d0, d1 in zip(self.input_slice[:-1], self.input_slice[1:])]
        losses = [self.active_models[i].loss(prediction[i], y_tensors[i])[0] for i in range(len(prediction))]
        ortho_loss = self.cluster_model.loss()

        loss = sum(losses) / float(len(losses))
        self.log("ptl/train_loss", loss)
        self.log("ptl/train_ortho_loss", ortho_loss)
        return loss+ortho_loss

    def validation_step(self, batch, batch_idx):
        x, y = batch

        x = x.view(-1, x.shape[-1])
        y = y.view(-1, y.shape[-1])
        prediction = self(x)
        y_tensors = [y[:, d0:d1] for d0, d1 in zip(self.input_slice[:-1], self.input_slice[1:])]
        losses = [self.active_models[i].loss(prediction[i], y_tensors[i])[0] for i in range(len(prediction))]

        loss = sum(losses) / float(len(losses))
        ortho_loss = self.cluster_model.loss()

        self.log('ptl/val_loss', loss, prog_bar=True)
        self.log("ptl/val_ortho_loss", ortho_loss, prog_bar=True)
        return {"val_loss":loss+ortho_loss}

    def test_step(self, batch, batch_idx):
        x, y = batch

        x = x.view(-1, x.shape[-1])
        y = y.view(-1, y.shape[-1])
        prediction = self(x)
        y_tensors = [y[:, d0:d1] for d0, d1 in zip(self.input_slice[:-1], self.input_slice[1:])]
        losses = [self.active_models[i].loss(prediction[i], y_tensors[i])[0] for i in range(len(prediction))]

        loss = sum(losses) / float(len(losses))
        ortho_loss = self.cluster_model.loss()

        self.log('ptl/test_loss', loss, prog_bar=True)
        self.log("ptl/test_ortho_loss", ortho_loss, prog_bar=True)
        return {"test_loss":loss+ortho_loss}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        self.log("avg_val_loss", avg_loss)
        if avg_loss < self.best_val_loss:
            self.best_val_loss = avg_loss
            if self.current_epoch % self.save_period == 0:
                self.save_checkpoint(best_val_loss=self.best_val_loss.item())

    def save_checkpoint(self, best_val_loss:float=np.inf, checkpoint_dir=MODEL_PATH):
        path = os.path.join(checkpoint_dir, self.name)

        config = {
            "hidden_dim": self.hidden_dim,
            "k": self.k,
            "lr": self.learning_rate,
            "batch_size": self.batch_size,
            "keep_prob":self.keep_prob,
            "optimizer": self.opt,
            "scheduler": self.scheduler,
            "scheduler_param": self.scheduler_param,
            "basis_func":self.config["basis_func"],
        }

        model_paths = [m.save_checkpoint(best_val_loss=best_val_loss, checkpoint_dir=path) for m in self.models]
        model = {"config":config, "name":self.name, "model_paths":model_paths,
                 "input_dims":self.input_dims, "pose_labels":self.pose_labels,
                 "cluster_model":self.cluster_model.state_dict()
                 }

        if not os.path.exists(checkpoint_dir):
            os.mkdir(checkpoint_dir)
        path = os.path.join(checkpoint_dir, self.name)
        if not os.path.exists(path):
            os.mkdir(path)

        filePath = os.path.join(path, str(best_val_loss)+"."+str(self.k)+".pbz2")
        with bz2.BZ2File(filePath, "w") as f:
            pickle.dump(model, f)
        return filePath

    @staticmethod
    def load_checkpoint(filePath):
        with bz2.BZ2File(filePath, "rb") as f:
            obj = pickle.load(f)
        model = RBF_Ortho(config=obj["config"], name=obj["name"],
                    input_dims=obj["input_dims"], pose_labels=obj["pose_labels"])

        models = [MLP.load_checkpoint(path) for path in obj["model_paths"]]
        model.models = models
        model.cluster_model.load_state_dict(obj["cluster_model"])
        return model

    def configure_optimizers(self):
        optimizer = self.opt(self.parameters(), lr=self.learning_rate)
        if self.scheduler is not None:
            scheduler = self.scheduler(optimizer, **self.scheduler_param)
            return [optimizer], [scheduler]
        return optimizer

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size, pin_memory=True, num_workers=self.workers)

    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.batch_size, pin_memory=True, num_workers=self.workers)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size, pin_memory=True, num_workers=self.workers)

    @staticmethod
    def init_params(m):
        if type(m) == nn.Linear:
            nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(.01)

    def add_models(self, input_dims:list=None, pose_labels:list=None, freeze=False):
        n = len(self.models)+1
        if pose_labels is not None:
            self.models += [MLP(config=self.config, dimensions=[input_dims[i]], pose_labels=pose_labels[i],
                            name="M" + str(i+n), single_module=0) for i in range(len(input_dims))]
        else:
            self.models += [MLP(config=self.config, dimensions=[input_dims[i]],
                            name="M" + str(i+n), single_module=0) for i in range(len(input_dims))]
        if freeze:
            for model in self.active_models:
                model.freeze(True)
            self.active_models = self.models[n-1:]
            self.input_dims = input_dims
        else:
            self.active_models = self.models
            self.input_dims += input_dims

        self.input_slice = [0] + list(accumulate(add, self.input_dims))

In [25]:
mix_model_name = "RBF_R1-R2_Joint"

mix_model = RBF_Ortho(config=config, input_dims=[pose_dim, pose_dim2],
                 train_set=train_set_p, val_set=val_set_p, test_set=test_set_p,
                 name=mix_model_name)

# mix_model.active_models[0] = MLP_v2(config=config, dimensions=[pose_dim],
#                  pos_dim=feature_dims["pos"], rot_dim=feature_dims["rotMat2"], vel_dim=feature_dims["velocity"],
#                  train_set=train_set_p1, val_set=val_set_p1, test_set=test_set_p1, name=ae_name1)

# mix_model.active_models[1] = MLP_v2(config=config, dimensions=[pose_dim2],
#                  pos_dim=feature_dims2["pos"], rot_dim=feature_dims2["rotMat2"], vel_dim=feature_dims2["velocity"],
#                  train_set=train_set_p2, val_set=val_set_p2, test_set=test_set_p2, name=ae_name2)

mix_model.active_models[0].encoder.load_state_dict(ae1.encoder.state_dict())
mix_model.active_models[0].decoder.load_state_dict(ae1.decoder.state_dict())
# mix_model.active_models[1].encoder.load_state_dict(c_model_R1.encoder.state_dict())
# mix_model.active_models[1].decoder.load_state_dict(c_model_R1.decoder.state_dict())




<All keys matched successfully>

In [22]:
mix_model.cluster_model.to("cuda")
mix_model.cluster_model.centres.to("cuda")
mix_model.cluster_model.sigmas.to("cuda")


Parameter containing:
tensor([0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100, 0.

In [26]:
MAX_EPOCHS = 200

checkpoint_callback = ModelCheckpoint(monitor="avg_val_loss", save_top_k=3)
earlystopping = EarlyStopping(monitor="avg_val_loss", patience=10)
logger=TensorBoardLogger(save_dir="logs/", name=mix_model_name, version="0.1")

trainer = pl.Trainer(
    default_root_dir="/home/nuoc/Documents/MEX/src/motion_generation/checkpoints",
    gpus=1, precision=16,
    callbacks=[earlystopping],
    min_epochs=20,
    logger=logger,
    max_epochs=MAX_EPOCHS,
    stochastic_weight_avg=True
)


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.


In [27]:
trainer.fit(mix_model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type      | Params
--------------------------------------------
0 | cluster_model | RBF_Layer | 65.8 K
--------------------------------------------
65.8 K    Trainable params
0         Non-trainable params
65.8 K    Total params
0.263     Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7f5b46c76cd0> for <torch.optim.swa_utils.SWALR object at 0x7f5b46caef10>
  warnings.warn(*args, **kwargs)


1

In [28]:
trainer.test(mix_model)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.7910033464431763,
 'ptl/test_ortho_loss': 277.9932861328125,
 'test_loss': 278.8456115722656}
--------------------------------------------------------------------------------


[{'test_loss': 278.8456115722656,
  'ptl/test_loss': 0.7910033464431763,
  'ptl/test_ortho_loss': 277.9932861328125}]

In [41]:
# model_name = "RBF_SINGLE_R1_Z_INPUT"
# model_name = "RBF_SINGLE_R1_Z_CONCAT_Joint_Ortho"
model_name = "RBF_SINGLE_R2_Z_CONCAT_Joint_Ortho"
# model_name = "RBF_SINGLE_R2_Z_CONCAT_Joint"
# model_name = "RBF_SINGLE_R2_Z_CONCAT_Joint"
# model_name = "RBF_SINGLE_R2_Z_INPUT_REDUCED"
# model_name = "AE_R2_REDUCED"

featureDim = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim,
    # "pose_dim": pose_dim2,
    # "pose_dim": pose_dim3,
    # "pose_dim": pose_dim4,
    "cost_dim": cost_dim,
    "g_input_dim": config["z_dim"] + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim,
    "pos_dim":feature_dims["pos"],
    # "pos_dim":feature_dims2["pos"],
    "rot_dim":feature_dims["rotMat2"],
    # "rot_dim":feature_dims2["rotMat2"],
    "vel_dim":feature_dims["velocity"],
    # "vel_dim":feature_dims2["velocity"],
    "posCost":feature_dims["posCost"],
    "rotCost":feature_dims["rotCost"]
    }

in_slice = [phase_dim, pose_dim, cost_dim]
# in_slice = [phase_dim, pose_dim2, cost_dim]
# in_slice = [phase_dim, pose_dim3, cost_dim]
# in_slice = [phase_dim, pose_dim4, cost_dim]
out_slice = [phase_dim, config["k"], cost_dim]

temp = RBF_Ortho(config=config, input_dims=[pose_dim, pose_dim2])

# pose_encoder = temp.active_models[0]
pose_encoder = temp.active_models[1]
# pose_encoder = temp.active_models[2]
# pose_encoder = temp.active_models[3]

# pose_encoder.encoder.load_state_dict(mix_model.active_models[0].encoder.state_dict())
# pose_encoder.decoder.load_state_dict(mix_model.active_models[0].decoder.state_dict())

# pose_encoder.encoder.load_state_dict(ae2.encoder.state_dict())
# pose_encoder.decoder.load_state_dict(ae2.decoder.state_dict())

pose_encoder.encoder.load_state_dict(mix_model.active_models[1].encoder.state_dict())
pose_encoder.decoder.load_state_dict(mix_model.active_models[1].decoder.state_dict())
# pose_encoder.encoder.load_state_dict(ae_temp.encoder.state_dict())
# pose_encoder.decoder.load_state_dict(ae_temp.decoder.state_dict())

# pose_encoder.encoder.load_state_dict(c_model_R2.encoder.state_dict())
# pose_encoder.decoder.load_state_dict(c_model_R2.decoder.state_dict())

# pose_encoder.encoder[3].requires_grad_(False)
# pose_encoder.encoder[5].requires_grad_(False)
# pose_encoder.encoder.load_state_dict(MLPMIX.active_models[0].encoder.state_dict())
# pose_encoder.decoder.load_state_dict(MLPMIX.active_models[0].decoder.state_dict())

# pose_encoder.encoder.load_state_dict(MLPMIX.active_models[1].encoder.state_dict())
# pose_encoder.decoder.load_state_dict(MLPMIX.active_models[1].decoder.state_dict())

# pose_encoder.encoder.load_state_dict(MLPMIX.active_models[2].encoder.state_dict())
# pose_encoder.decoder.load_state_dict(MLPMIX.active_models[2].decoder.state_dict())

# pose_encoder.encoder.load_state_dict(MLPMIX.active_models[3].encoder.state_dict())
# pose_encoder.decoder.load_state_dict(MLPMIX.active_models[3].decoder.state_dict())

# middle_layer = torch.nn.Sequential()
middle_layer = temp.cluster_model
middle_layer.load_state_dict(mix_model.cluster_model.state_dict())
middle_layer.requires_grad_(False)
# cost_encoder = model2.cost_encoder

# generation_model = model2.generationModel
model = MotionGenerationModel_v4(config=config, Model=MoE, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim,
                                 input_slicers=in_slice, output_slicers=out_slice,
                                 # train_set=t1, val_set=v1, test_set=te1,
                                 train_set=t2, val_set=v2, test_set=te2,
                                 # train_set=t3, val_set=v3, test_set=te3,
                                 # train_set=t4, val_set=v4, test_set=te4,
                                 name=model_name
                                   )



In [57]:
middle_layer = temp.cluster_model
model3 = MotionGenerationModel_v4.load_checkpoint(
    "/home/nuoc/Documents/MEX/models/version_0.2/MLPMIX_COSINE_withLabel_MLP_MOE_R2_TWO_REDUCED/0.2841307520866394.pbz2",
    Model=MoE, MiddleModel=middle_layer
)

In [42]:
model.generationModel.gate.load_state_dict(model1.generationModel.gate.state_dict())
model.generationModel.load_state_dict(model1.generationModel.state_dict())

# model.generationModel.freeze()
# model.cost_encoder.freeze()
# model.middle_layer.requires_grad_(False)

<All keys matched successfully>

In [43]:

MAX_EPOCHS = 100

checkpoint_callback = ModelCheckpoint(monitor="avg_val_loss", save_top_k=3)
earlystopping = EarlyStopping(monitor="avg_val_loss", patience=20)
logger=TensorBoardLogger(save_dir="logs/", name=model_name, version="0.1")

trainer = pl.Trainer(
    default_root_dir="/home/nuoc/Documents/MEX/src/motion_generation/checkpoints",
    gpus=1, precision=16,
    # callbacks=[earlystopping],
    min_epochs=20,
    logger=logger,
    max_epochs=MAX_EPOCHS,
    stochastic_weight_avg=True
)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.


In [44]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type      | Params
-----------------------------------------------
0 | pose_autoencoder | MLP       | 454 K 
1 | middle_layer     | RBF_Layer | 65.8 K
2 | cost_encoder     | MLP       | 36.2 K
3 | generationModel  | MoE       | 2.4 M 
-----------------------------------------------
2.9 M     Trainable params
65.8 K    Non-trainable params
3.0 M     Total params
12.018    Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7f5afc7c2c10> for <torch.optim.swa_utils.SWALR object at 0x7f5b3c330790>
  warnings.warn(*args, **kwargs)


1

In [45]:
trainer.test(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.2379748374223709,
 'ptl/test_min_pos_cost': -0.009179978631436825,
 'ptl/test_min_rot_cost': 0.5173352360725403,
 'ptl/test_min_target_pos_cost': nan,
 'ptl/test_min_target_rot_cost': nan,
 'ptl/test_pos_loss': 0.0352429635822773,
 'ptl/test_rot_loss': 0.08676347136497498,
 'ptl/test_sum_pos_cost': -0.0667581707239151,
 'ptl/test_sum_rot_cost': 6.3550639152526855,
 'ptl/test_sum_target_pos_cost': nan,
 'ptl/test_sum_target_rot_cost': nan,
 'test_loss': 0.28225940465927124}
--------------------------------------------------------------------------------


[{'test_loss': 0.28225940465927124,
  'ptl/test_loss': 0.2379748374223709,
  'ptl/test_pos_loss': 0.0352429635822773,
  'ptl/test_rot_loss': 0.08676347136497498,
  'ptl/test_min_pos_cost': -0.009179978631436825,
  'ptl/test_min_target_pos_cost': nan,
  'ptl/test_min_rot_cost': 0.5173352360725403,
  'ptl/test_min_target_rot_cost': nan,
  'ptl/test_sum_pos_cost': -0.0667581707239151,
  'ptl/test_sum_target_pos_cost': nan,
  'ptl/test_sum_rot_cost': 6.3550639152526855,
  'ptl/test_sum_target_rot_cost': nan}]

In [33]:
# model1 = model # ~0.244, 0.16, 0.136, 0.22 (ortho)
model2 = model # ~0.33, 0.277, 0.16, 0.20 (AE), 0.28 (ortho)

# model2_1 = model  # ~0.34, No resuing Mogen
# model2_1 = model  # ~0.30, Reusing everything
# model2_2 = model  # ~0.22 Reusing Mogen
# model2_3 = model   # ~0.2 Reusing Mogen no C
# model2_3 = model   # ~0.16



In [46]:
# te = te11
te = te22
# te = te33
# te = te4
# te = test_set_p1
n = 5
# idx = np.random.randint(0, len(te), n)
idx = np.arange(n)
original = []
generated = []
# pose_idx_upper = feature_dims2["phase_dim"] + feature_dims["pos"] + feature_dims["rotMat2"]
# pose_idx_upper = model.in_slices[1] + feature_dims["pos"] + feature_dims["rotMat2"]
# print(pose_idx_upper)


In [47]:
# model = model2_1
# model = model2_1
# model = model2_2
# model = model3

# model.autoregress_prob = 1
with torch.no_grad():
    model.eval()
    model.cpu()
    # for i in range(1):
    # original =
    # x = x_tensors[idx]
    x = torch.stack([te[i][0] for i in idx])
    y = torch.stack([te[i][1] for i in idx])
    shape = x.shape
    x = x.view(-1, shape[-1])

    # x = x.view(-1, 13, shape[-1])
    # x_c = x[:,0,:]
    # n = shape[1]
    g_frames = []
    #
    # for i in range(0, 13):
    x.cpu()
    out= model(x)
    x_c = torch.cat(out,dim=1).detach()
    # g_frames.append(x_c.unsqueeze(1))

    # out = torch.cat(model(x), dim=1).view(shape)
            # x_c = torch.cat(out, dim=1)
    # g_frames.append(x_c.unsqueeze(1))
        # original.append(o_frames)
    # generated.append(torch.cat(g_frames, dim=1))
    generated = x_c
    # generated = out
    # generated = torch.cat(g_frames, dim=1)
    generated = generated.view(shape)





In [48]:
print(y.size())
print(generated.size())

torch.Size([5, 299, 452])
torch.Size([5, 299, 404])


In [49]:
phase= feature_dims["phase_vec_l2"]
# phase_dim = 0
toPosDim = phase+feature_dims["pos"]
toRotDim = toPosDim + feature_dims["rotMat2"]
toVelDim = toRotDim + feature_dims["velocity"]

gPos = generated[:, :, phase:toPosDim]
gRot = generated[:, :, toPosDim:toRotDim]
gVel = generated[:, :, toRotDim:toVelDim]

oPos = y[:, :, phase:toPosDim]
oRot = y[:, :, toPosDim:toRotDim]
oVel = y[:, :, toRotDim:toVelDim]

tPos = y[:, :, -target_dim:-target_dim+3*4]
tRot = y[:, :, -target_dim+3*4:]

print(gPos.shape, gRot.shape, gVel.shape)
print(oPos.shape, oRot.shape, oVel.shape)
# print(tPos.shape, tRot.shape)


torch.Size([5, 299, 93]) torch.Size([5, 299, 186]) torch.Size([5, 299, 93])
torch.Size([5, 299, 93]) torch.Size([5, 299, 186]) torch.Size([5, 299, 93])


In [50]:
clip_length = gPos.shape[1]
gPos_r = gPos.reshape((n, clip_length, -1, 3))
gRot_r = gRot.reshape((n, clip_length, -1, 3, 2))
gVel_r = gVel.reshape((n, clip_length, -1, 3))

oPos_r = oPos.reshape((n, clip_length, -1, 3))
oRot_r = oRot.reshape((n, clip_length, -1, 3, 2))
oVel_r = oVel.reshape((n, clip_length, -1, 3))

tPos_r = tPos.reshape((n, clip_length, -1, 3))
tRot_r = tRot.reshape((n, clip_length, -1, 3, 3))

print("Pos loss: ", torch.nn.functional.mse_loss(gPos_r, oPos_r))
print("Rot loss: ", torch.nn.functional.mse_loss(gRot_r, oRot_r))
print("Vel loss: ", torch.nn.functional.mse_loss(gVel_r, oVel_r))



Pos loss:  tensor(0.0365)
Rot loss:  tensor(0.0881)
Vel loss:  tensor(0.2404)


In [39]:
template_R1= js.load(open("/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/TestAll_1_R1_One_1/False_2_0.json"))
template_R2 = js.load(open("/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Test/R2.json"))

In [51]:
def setVec3(struct, vec):
    struct["x"] = vec[0].item()
    struct["y"] = vec[1].item()
    struct["z"] = vec[2].item()

def setVec6(struct, vec):
    for r, cell in enumerate(["x", "y", "z"]):
        for col, column in enumerate(["c0", "c1"]):
            struct[column][cell] = vec[r, col].item()

def insert_pos(template,
               positions=None, rotations=None, velocity=None,
               tPos=None, tRot=None, name="Replay"):
    shape = positions.shape
    for c in range(shape[0]):
        for f in range(shape[1]):
            t = 0
            for j in range(shape[2]):
                jo = template["frames"][f]["joints"][j]
                if positions is not None:
                    setVec3(jo["position"], positions[c,f,j])
                if rotations is not None:
                    setVec3(jo["velocity"], velocity[c,f,j])
                if velocity is not None:
                    setVec6(jo["rotMat"], rotations[c,f,j])

                # jo["position"]["x"] = positions[c,f,j,0].item()
                # jo["position"]["y"] = positions[c,f,j,1].item()
                # jo["position"]["z"] = positions[c,f,j,2].item()

                # jo["velocity"]["x"] = velocity[c,f,j,0].item()
                # jo["velocity"]["y"] = velocity[c,f,j,1].item()
                # jo["velocity"]["z"<] = velocity[c,f,j,2].item()
                #
                # for r, cell in enumerate(["x", "y", "z"]):
                #     for col, column in enumerate(["c0", "c1"]):
                #         jo["rotMat"][column][cell] = rotations[c,f,j,r, col].item()

                if jo["key"]:
                    if tPos is not None:
                        setVec3(jo["cost"]["TargetPosition"], tPos[c,f,t])
                    if tRot is not None:
                        setVec6(jo["cost"]["TargetRotation"], tRot[c,f,t])
                    t+=1
                    # jo["targetPosition"]["x"] = tPos[c, f, t, 0]
                    # jo["targetPosition"]["y"] = tPos[c, f, t, 1]
                    # jo["targetPosition"]["z"] = tPos[c, f, t, 2]

                    # for r, cell in enumerate(["x", "y", "z"]):
                    #     for col, column in enumerate(["c0", "c1"]):
                    #         jo["rotMat"][column][cell] = rotations[c,f,j,r, col].item()
        with open("{}_{}.json".format(name, c), "w") as f:
            js.dump(template, f)

# os.mkdir("/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/R1_ALL")
# insert_pos(template_R1, oPos_r, oRot_r, oVel_r, tPos_r, tRot_r,
#            "/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/R1_AE/RBF_R1_Concat_Joint_Ortho_Original")
# insert_pos(template_R1, gPos_r, gRot_r, oVel_r, tPos_r, tRot_r,
#            "/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/R1_AE/RBF_R1_Concat_Joint_Ortho_Generated")
insert_pos(template_R2, oPos_r, oRot_r, oVel_r, tPos_r, tRot_r,
           "/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation//R1_AE/RBF_R2_CONCAT_JOINT_Ortho_REDUCED_Original")
insert_pos(template_R2, gPos_r, gRot_r, oVel_r, tPos_r, tRot_r,
           "/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation//R1_AE/RBF_R2_CONCAT_JOINT_Ortho_REDUCED_Generated")

# insert_pos(template_R2, oPos_r, oRot_r, oVel_r, None, None,
#            "/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/R1_AE/R2_v4_Original")
# insert_pos(template_R2, gPos_r, gRot_r, oVel_r, None, None,
#            "/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/R1_AE/R2_v4_Generated")